In [2]:
import pandas as pd
import numpy as np
from lreg import LogisticRegression as lr
from sklearn.preprocessing import LabelEncoder
from sklearn import datasets
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import time

In [3]:
iris = datasets.load_iris()
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])
idf = df[df.target.isin([1,2])]

In [4]:
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [5]:

scaler = StandardScaler()

cols_to_norm = [
'sepal length (cm)',
'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)'
 ]
# X = idf.copy()

# idf.hist(column='sepal length (cm)',bins=100) # before scaling

X = pd.DataFrame(scaler.fit_transform(idf[cols_to_norm]),columns=cols_to_norm)
# dff.hist(column='sepal length (cm)',bins=100) # after scaling

X = X.apply(pd.to_numeric)

In [17]:
le = LabelEncoder()
le.fit( idf['target'] )
y = pd.Series( data = le.transform( idf['target'] ) )

test_s = 50
train_s = 50

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_s, train_size=train_s, random_state=42)

In [18]:
model = lr()
model.fit( X_train, y_train, lr=0.01, max_iter=1e4, method='sgd' )
accuracy_score(y_test,  model.predict(X_test).round())

0.92

In [19]:
model = lr()
start = time.time()
model.fit( X_train, y_train, lr=0.01, max_iter=1e4, method='sgd' )
print(time.time() - start)

print('\nscore:')
from sklearn.metrics import accuracy_score
accuracy_score(y_test,  np.around(model.predict(X_test)))

4.5395965576171875

score:


0.92

In [20]:
model = lr()
start = time.time()
model.fit( X_train, y_train, lr=0.01, max_iter=1e4, method='momentum', hist_w=0. )
print(time.time() - start)

print('\nscore:')
from sklearn.metrics import accuracy_score
accuracy_score(y_test,  np.around(model.predict(X_test)))

5.042074680328369

score:


0.92

In [21]:
model = lr()
start = time.time()
model.fit( X_train, y_train, lr=0.01, max_iter=1e4, method='rmsprop', hist_w=0. )
print(time.time() - start)

print('\nscore:')
from sklearn.metrics import accuracy_score
accuracy_score(y_test,  np.around(model.predict(X_test)))

5.779108762741089

score:


0.9

In [22]:
model = lr()
start = time.time()
model.fit( X_train, y_train, lr=0.01, max_iter=1e4, method='nag', hist_w=0. )
print(time.time() - start)

print('\nscore:')
from sklearn.metrics import accuracy_score
accuracy_score(y_test,  np.around(model.predict(X_test)))

6.4147560596466064

score:


0.92

In [23]:
# Контрольный расчет: проверить, что ничего основного не сломано - numply_ml без изменений, sgd
from lreg_numpyml_pure import LogisticRegression as lr_p
model = lr_p()
start = time.time()
model.fit( X_train, y_train, lr=0.01, max_iter=1e4)
print(time.time() - start)

print('\nscore:')
from sklearn.metrics import accuracy_score
accuracy_score(y_test,  np.around(model.predict(X_test)))

5.093043327331543

score:


0.92